In [4]:
import pandas as pd
import numpy as np
import glob

from matplotlib import pyplot as plt
import seaborn as sns

import re
import string

from tqdm import tqdm

import transformers
# from transformers import BertModel, BertConfig, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import networkx as nx
from node2vec import Node2Vec

## Merging all csv files

In [5]:
filenames = glob.glob("russian-troll-tweets/*.csv")

li = []
for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
dfs = pd.concat(li, axis=0, ignore_index=True)

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,15,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
dfs.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


In [7]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946207 entries, 0 to 2946206
Data columns (total 21 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   external_author_id  object
 1   author              object
 2   content             object
 3   region              object
 4   language            object
 5   publish_date        object
 6   harvested_date      object
 7   following           int64 
 8   followers           int64 
 9   updates             int64 
 10  post_type           object
 11  account_type        object
 12  retweet             int64 
 13  account_category    object
 14  new_june_2018       int64 
 15  alt_external_id     object
 16  tweet_id            int64 
 17  article_url         object
 18  tco1_step1          object
 19  tco2_step1          object
 20  tco3_step1          object
dtypes: int64(6), object(15)
memory usage: 472.0+ MB


## Keeping English tweets

In [8]:
dfs = dfs[dfs['language'] == 'English']
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2116867 entries, 0 to 2946181
Data columns (total 21 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   external_author_id  object
 1   author              object
 2   content             object
 3   region              object
 4   language            object
 5   publish_date        object
 6   harvested_date      object
 7   following           int64 
 8   followers           int64 
 9   updates             int64 
 10  post_type           object
 11  account_type        object
 12  retweet             int64 
 13  account_category    object
 14  new_june_2018       int64 
 15  alt_external_id     object
 16  tweet_id            int64 
 17  article_url         object
 18  tco1_step1          object
 19  tco2_step1          object
 20  tco3_step1          object
dtypes: int64(6), object(15)
memory usage: 355.3+ MB


In [9]:
dfs['account_type'].value_counts()

Right         704953
local         458012
Left          422141
Hashtager     236092
news          138581
Commercial    112580
Russian        20015
Koch           10759
?               6945
German          3567
Italian         1799
Arabic          1030
ZAPOROSHIA       172
Spanish           79
Ebola             70
French            70
Portuguese         2
Name: account_type, dtype: int64

In [10]:
dfs['account_category'].value_counts()

RightTroll      704953
NewsFeed        596593
LeftTroll       422141
HashtagGamer    236092
Commercial      112580
NonEnglish       26562
Fearmonger       11001
Unknown           6945
Name: account_category, dtype: int64

# Creating 'hashtags' and 'mentions'  columns

### temp_df with 'author' and 'content'

In [11]:
temp_df = dfs[['author', 'content']]
temp_df.head()

,author,content
0,10_GOP,"""We have a sitting Democrat US Senator on tria..."
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...
3,10_GOP,JUST IN: President Trump dedicates Presidents ...
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF..."


### Drop NaN

In [12]:
temp_df.dropna(how='any', inplace=True)
print(temp_df.shape)

(2116866, 2)


C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## 'hashtags' column

- #### Collecting hashtags from content

In [13]:
temp_df['hashtags'] = dfs['content'].str.findall(r'#.*?(?=\s|$)')
temp_df.head()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,author,content,hashtags
0,10_GOP,"""We have a sitting Democrat US Senator on tria...",[]
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,[]
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,[#BoycottNFL]
3,10_GOP,JUST IN: President Trump dedicates Presidents ...,[]
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",[#StandForOurAnthem🇺🇸]


## 'mentions' column

- #### Collecting mentions from content

In [14]:
temp_df['mentions'] = dfs['content'].str.findall(r'@.*?(?=\s|$)')
temp_df.head()

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,author,content,hashtags,mentions
0,10_GOP,"""We have a sitting Democrat US Senator on tria...",[],[@nedryun]
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,[],[]
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,[#BoycottNFL],[]
3,10_GOP,JUST IN: President Trump dedicates Presidents ...,[],[]
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",[#StandForOurAnthem🇺🇸],[]


# Graph requirements

- U2H
 + dictionary contains {'author' : list of all his unique hashtags}
 + List of unique authors
 + List of unique hashtags
 
- U2M
 + dictionary contains {'author' : list of all his unique mentions}
 + List of unique authors
 + List of unique mentions

## U2H

#### - Creating dictionary contains {'author' : list of all his unique hashtags}

In [15]:
hashtags_dic = dict()
for i in tqdm(range(temp_df.shape[0]), position=0):
    hashtags_dic.setdefault(temp_df.iloc[i, 0], []).append(temp_df.iloc[i, 2])

100%|█████████████████████████████████████████████████████████████████████| 2116866/2116866 [00:27<00:00, 77516.46it/s]


In [16]:
print("author: ", list(hashtags_dic.keys())[0])
print("\nhashtags: ", list(hashtags_dic.values())[0])
print("\n# hashtags:", len(list(hashtags_dic.values())[0]))

author:  10_GOP

hashtags:  [[], [], ['#BoycottNFL'], [], ['#StandForOurAnthem🇺🇸'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['#HipHopAwards'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['#Iran'], [], [], ['#MAGA'], [], [], [], ['#WomenBoycottTwitter'], [], [], [], [], [], [], [], [], [], [], ['#FakeNews'], [], [], [], [], ['#LasVegas'], ['#LasVegas.'], [], ['#LasVegasShooting'], [], ['#LasVegasStrong🇺🇸'], ['#LasVegasShooting'], [], [], ['#VegasStrong'], [], ['#VegasStrong'], ['#BoycottNFL'], ['#SanJuan', '#TakeAKnee'], [], ['#FakeNews'], [], [], [], [], ['#PRStrong'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['#NFLBoycott', '#MAGA'], [], [], [], [], [], [], [], [], ['#IrmaHurricane2017'], ['#HurrcaneIrma'], [], ['#FightCensorship', '#FreeSpeech', '#LiberalAttack'], ['#HurricaneIrma', '#Irma', '#TampaReady', '#FLORIDAHURRICANE'], ['#Irma', '#IrmaHurricane2017'], [], ['#HurricaneIrma', '#HurrcaneIrma'], 

#### - Flatting the hashtags lists for each author

In [17]:
for i, (k, v) in enumerate(hashtags_dic.items()):
    hashtags_dic[k] = set([item for sublist in list(hashtags_dic.values())[i] for item in sublist])

In [18]:
print("author: ", list(hashtags_dic.keys())[0])
print("\nhashtags: ", list(hashtags_dic.values())[0])
print("\n# hashtags:", len(list(hashtags_dic.values())[0]))

author:  10_GOP

hashtags:  {'#DACADeal', '#HurrcaneIrma', '#LasVegas', "#MAGA'", '#Irma', '#dotard', '#1A', '#Manafort', '#ConstitutionDay', '#IRMA', '#FakeNews', '#NeverForget911', '#FightCensorship', '#MOAR.', '#EMMY2017', '#NeverForget', '#60Minutes', '#ColinKaepernick:', '#FLORIDAHURRICANE', '#Maddow', '#BoycottNFL', '#PrayForMexico', '#JJNonPoliticalTweet', '#WeStand', '#September11th', '#dogs', '#HerpesHillary', '#VegasStrong', '#TaxReform.', "#AmericaFirst'", '#TeamScalise', '#SanJuan', '#Iran', '#NFLBoycott', '#CNNisFakeNews', '#TampaReady', '#NRA', '#FreeSpeech', '#Berkeley', '#WomenBoycottTwitter', '#AfterIrma', '#MOAR', '#TrumpTrain', '#TuesdayThoughts', '#WhatHappened', '#BenAtBerkeley', '#MAGA', '#HurricaneIrma', '#IrmaHurricane2017', '#LasVegas.', '#TropicalStormIrma', '#StandForOurAnthem🇺🇸', '#LasVegasStrong🇺🇸', '#PRStrong', '#LiberalAttack', '#FEMA', '#CrookedHillary', '#TakeAKnee', '#HipHopAwards', '#LasVegasShooting', '#IfIWerePresidentForOneDay', '#STLVerdict', '#Fr

#### - Creating a list with all unique hashtags

In [19]:
hashtags_list = [item for sublist in list(temp_df['hashtags']) for item in sublist]

In [20]:
print(len(hashtags_list))
unique_hashtags_list = set(hashtags_list)
print(len(unique_hashtags_list))

1457604
110097


## U2M

#### - Creating dictionary contains {'author' : list of all his unique mentions}

In [21]:
mentions_dic = dict()
for i in tqdm(range(temp_df.shape[0]), position=0):
    mentions_dic.setdefault(temp_df.iloc[i, 0], []).append(temp_df.iloc[i, 3])

100%|█████████████████████████████████████████████████████████████████████| 2116866/2116866 [00:28<00:00, 75414.76it/s]


In [22]:
print("author: ", list(mentions_dic.keys())[0])
print("\nhashtags: ", list(mentions_dic.values())[0])
print("\n# hashtags:", len(list(mentions_dic.values())[0]))

author:  10_GOP

hashtags:  [['@nedryun'], [], [], [], [], [], [], ['@SenatorMenendez', '@CarmenYulinCruz'], [], ['@CNN'], ['@CNN'], ['@thehill'], [], ['@MichelleObama,'], ['@MichelleObama,'], [], ['@FLOTUS!'], [], ['@Breaking911'], [], [], [], [], [], [], [], [], [], [], [], [], ['@realDonaldTrump'], ['@realDonaldTrump', '@POTUS'], [], [], [], [], [], [], [], [], [], [], [], ['@Joy_Villa'], [], [], [], [], ['@Joy_Villa:'], [], [], [], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], ['@realDonaldTrump'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['@Cernovich', '@KFILE'], [], [], [], [], [], [], [], [], ['@CNN'], [], ['@AP_Politics'], ['@POTUS', '@FLOTUS'], [], [], [], [], [], ['@PrisonPlanet'], ['@PrisonPlanet', '@GrrrGraphics,one'], [], [], [], [], [], [], [], [], [], [], ['@YouTube', '@TeamYouTube,'], ['@SteveScalise'], [], [], [], [], [], [], [], [], ['@10_gop'], [], [], ['@10_gop'], []

#### - Flatting the mentions lists for each author

In [23]:
for i, (k, v) in enumerate(mentions_dic.items()):
    mentions_dic[k] = set([item for sublist in list(mentions_dic.values())[i] for item in sublist])

In [24]:
print("author: ", list(mentions_dic.keys())[0])
print("\nhashtags: ", list(mentions_dic.values())[0])
print("\n# hashtags:", len(list(mentions_dic.values())[0]))

author:  10_GOP

hashtags:  {'@realTEN_GOP', '@KFILE', '@10_gop.', '@Joy_Villa:', '@FLOTUS', '@CNN', '@umknicken', '@tamaraleighllc', '@brianstelter,', '@realDonaldTrump', '@GrrrGraphics,one', '@nedryun', '@POTUS', '@SenatorMenendez', '@DonnaShelley2', '@FoxNews', '@RedPillBlack:', '@LauraLoomer', '@thehill', '@PrisonPlanet', '@Marines4MAGA', '@TEN_GOP', '@ShopFloorNAM', '@UN', '@JackPosobiec', '@5sahandful', '@Breaking911', '@Cernovich', '@MichelleObama,', '@10_gop', '@HillaryClinton', '@ChaVonZee16', '@netanyahu', '@SLMPD', '@CarmenYulinCruz', '@Thewall77515902', '@jojoh888', '@TeamYouTube,', '@SteveScalise!', '@FLOTUS!', '@Roni_K_Patriot', '@SebGorka', '@exposes_racism', '@benshapiro', '@Joy_Villa', '@SteveScalise', '@NaeguNugu', '@larryelder', '@YouTube', '@ELEVEN_GOP', '@AP_Politics', '@GIT_ER_DONE_USA', '@wahrbear'}

# hashtags: 53


#### - Creating a list with all unique mentions

In [25]:
mentions_list = [item for sublist in list(temp_df['mentions']) for item in sublist]

In [26]:
print(len(mentions_list))
unique_mentions_list = set(mentions_list)
print(len(unique_mentions_list))

1043278
566628


# Contructing U2H and U2M graphs

In [27]:
import networkx as nx
U2H = nx.Graph()
U2M = nx.Graph()

### Adding nodes

#### -  'author' nodes

In [28]:
U2H.add_nodes_from(dic.keys())
U2M.add_nodes_from(dic.keys())

NameError: name 'dic' is not defined

#### - 'hashtags' and 'mentions' nodes

In [ ]:
U2H.add_nodes_from(unique_hashtags_list)
U2M.add_nodes_from(unique_hashtags_list)

### Creating edges

#### - U2H edges

In [ ]:
for i, (k, v) in enumerate(dic.items()):
    for h in v:
        U2H.add_edge(k, h)

#### - U2M edges

In [ ]:
for i, (k, v) in enumerate(mentions_dic.items()):
    for h in v:
        U2M.add_edge(k, h)

# Fitting U2H and U2M graphs
We give node2vec.Node2Vec a networkx.Graph instance, and after using .fit() (which accepts any parameter accepted by we get a gensim.models.Word2Vec) we get in return a gensim.models.Word2Vec instance.

## U2H

### - Creating node2vec instance

In [ ]:
# Generate walks
node2vec = Node2Vec(G, dimensions=128, walk_length=10, num_walks=10)

### - Fitting the model

In [ ]:
# Learn embeddings 
model = node2vec.fit(window=10, min_count=1)

### - Inspect the similarity

In [ ]:
for node, _ in model.most_similar('#Palestinians'):
    print(node)

## U2M

### - Creating node2vec instance

In [ ]:
# Generate walks
U2M_node2vec = Node2Vec(U2M, dimensions=128, walk_length=10, num_walks=10)

### - Fitting the model

In [ ]:
# Learn embeddings 
U2M_model = U2M_node2vec.fit(window=10, min_count=1)

### - Inspect the similarity

In [ ]:
for node, _ in U2M_model.most_similar('@TeamYouTube'):
    print(node)